### Optout model

### 01. Packages and connection

In [1]:
import pandas as pd
from pyathena import connect
import matplotlib.pyplot as plt
from functools import reduce

# create a connection to Athen
conn = connect(s3_staging_dir = 's3://aws-athena-query-results-341377015103-eu-west-2/',
                   region_name='eu-west-2') 

pd.options.display.max_columns = 500

### 02. Base and target

- get all the customers who opted out any point of time
- Out of Sample period will be seperated in the model
- I reduce the time frame so instead of a full year i just have data from the beining of the year this will help reduce 
- the problem i have with the other quires that run out of scalar


In [2]:
df = pd.read_sql('''select * from campaign_data.atc_sccv_ids limit 100''', conn)
df.columns

Index(['comm_id', 'brand', 'bt_customer_id', 'ee_customer_id',
       'date_of_delivery', 'open_flg', 'open_cnt', 'open_dttm', 'open_date',
       'click_flg', 'click_cnt', 'click_dttm', 'click_date', 'optout_flg',
       'optout_date', 'control_grp_flg', 'campaign_code', 'campaign_name',
       'campaign_objective', 'business_objective', 'product_objective',
       'product_offer', 'product_offer_name', 'target_base', 'campaign_type',
       'campaign_tier', 'campaign_freequency', 'campaign_target_type',
       'channel', 'trigger_flag', 'delivery_stage', 'model_used_flag',
       'model_name', 'percentile', 'planned_start_dt', 'planned_end_dt',
       'first_time_response', 'last_time_response',
       'days_to_respond_first_time', 'days_to_respond_last_time',
       'customer_segment', 'customer_subsegment', 'marketing_cohort',
       'marketing_cohort_base', 'marketing_cohort_device',
       'marketing_cohort_lifestage', 'customer_mrc', 'customer_current_arpu',
       'customer_con

In [3]:
#New data

sql_code = '''

select 
count(*) ,sum(optout_flg)  
from campaign_data.atc_sccv_ids

where brand='EE'
and channel in ('M','E')
and control_grp_flg = 'N'
and campaign_objective = 'CROSS-SELL'
-- and cast(date_format(date_of_delivery,'%Y-%m-%d') as date)  < date_parse('2020-12-31','%Y-%m-%d')
and cast(date_format(date_of_delivery,'%Y-%m-%d') as date)  > date_parse('2020-01-01','%Y-%m-%d')

'''

df = pd.read_sql(sql_code, conn)
df.head()

,_col0,_col1
0,57278713,26221


In [4]:
sql_code = '''
select campaign_code,cnt,oo_cnt,oor from (
    select 
    campaign_code,
    count(distinct ee_customer_id) as cnt,
    sum(optout_flg) as oo_cnt,
    cast(sum(optout_flg) as decimal(15,5))/cast(count(distinct ee_customer_id) as decimal(15,5)) as oor
    from campaign_data.atc_sccv_ids

    where brand='EE'
    and channel in ('M','E')
    and control_grp_flg = 'N'
    and campaign_objective = 'CROSS-SELL'
    -- and cast(date_format(date_of_delivery,'%Y-%m-%d') as date)  < date_parse('2020-12-31','%Y-%m-%d')
    and cast(date_format(date_of_delivery,'%Y-%m-%d') as date)  > date_parse('2020-01-01','%Y-%m-%d')
    group by 1)
where cnt > 75000 and cnt < 2000000 -- and oor > 0.0001 
'''

df = pd.read_sql(sql_code, conn)
df#.head()


,campaign_code,cnt,oo_cnt,oor
0,DR0000015757,1776221,841,0.00047
1,DR0000015656,1112913,608,0.00055
2,DR0000015902,771670,112,0.00015
3,DR0000015620,1220956,455,0.00037
4,DR0000015738,1961846,1410,0.00072
5,DR0000015969,338627,9,0.00003
6,DR0000015777,1086743,402,0.00037
7,DR0000015833,1889422,1280,0.00068
8,DR0000015918,392149,58,0.00015
9,DR0000011515,849253,1280,0.00151


In [5]:
109+1410+9+34+550+24+1280+788+480+112+608+58+329

5791

# NOTES

- Globally - INFORMING categories are supposed to be service (but historically have been used for some marketing)
- OOCN - Many of these large service campaigns were sent under 'upgrade and retention'


In [6]:
#base of customers who have opted out
sql_code = '''
drop table if exists  campaign_data.atc_ee_optout_customers_model;
'''
pd.read_sql(sql_code, conn)

sql_code = '''
CREATE TABLE IF NOT EXISTS campaign_data.atc_ee_optout_customers_model as 

select 
    -- Build joining dates
    *
    ,DATE_ADD('day',-6,cast(date_format(optout_date,'%Y-%m-%d') as date)) as day_1_week_ago -- CLM
    ,format_datetime(date_add('month',-1,cast(date_format(optout_date,'%Y-%m-%d') as date) ),'yMM') as date_month -- AGG
from (
    -- Identify opt out base
    select 
        ee_customer_id,
        max(optout_flg) as optout_flag,
        max(click_flg) as click_flag,           
        sum(optout_flg) as optout_cnt,   
        min(date_of_delivery) as optout_date 
    from campaign_data.atc_sccv_ids base
    -- inner to filter by size    
    inner join (
        select campaign_code,cnt,oo_cnt,oor from (
            select 
            campaign_code,
            count(distinct ee_customer_id) as cnt,
            sum(optout_flg) as oo_cnt,
            cast(sum(optout_flg) as decimal(15,5))/cast(count(distinct ee_customer_id) as decimal(15,5)) as oor
            from campaign_data.atc_sccv_ids

            where brand='EE'
            and channel in ('M','E')
            and control_grp_flg = 'N'
            and campaign_objective = 'CROSS-SELL'
            -- and cast(date_format(date_of_delivery,'%Y-%m-%d') as date)  < date_parse('2020-12-31','%Y-%m-%d')
            and cast(date_format(date_of_delivery,'%Y-%m-%d') as date)  > date_parse('2020-01-01','%Y-%m-%d')
            group by 1)
        where cnt > 75000 and cnt < 2000000 and oor > 0.0001 
        ) filter_by_size
        on base.campaign_code = filter_by_size.campaign_code
    
    where brand='EE'
    and channel in ('M','E')
    and control_grp_flg = 'N'
    and campaign_objective = 'CROSS-SELL'
    -- and cast(date_format(date_of_delivery,'%Y-%m-%d') as date)  < date_parse('2020-12-31','%Y-%m-%d')
    and cast(date_format(date_of_delivery,'%Y-%m-%d') as date)  > date_parse('2020-01-01','%Y-%m-%d')
    
    and optout_flg=1 

    group by ee_customer_id having sum(optout_flg)>0
    ) A
'''

pd.read_sql(sql_code, conn)

#QA
df = pd.read_sql('''select count(*),max(optout_flag) from campaign_data.atc_ee_optout_customers_model;''', conn)
df.head()

,_col0,_col1
0,9919,1


In [7]:
#Base of customers who have NOT opted out
sql_code = '''
drop table if exists  campaign_data.atc_ee_not_optout_customers_model;
'''
pd.read_sql(sql_code, conn)

sql_code = '''
CREATE TABLE IF NOT EXISTS campaign_data.atc_ee_not_optout_customers_model as 

select 
    -- Build joining dates
    * 
    ,DATE_ADD('day',-6,cast(date_format(optout_date,'%Y-%m-%d') as date)) as day_1_week_ago
    ,format_datetime(date_add('month',-1,cast(date_format(optout_date,'%Y-%m-%d') as date) ),'yMM') as date_month
from ( 
    -- sample non-opt out base    
    select 
        ee_customer_id,
        max(optout_flg) as optout_flag,
        max(click_flg) as click_flag,        
        sum(optout_flg) as optout_cnt,   
        max(date_of_delivery) as optout_date

    from campaign_data.atc_sccv_ids base
    -- inner to filter by size
    inner join (
        select campaign_code,cnt,oo_cnt,oor from (
            select 
            campaign_code,
            count(distinct ee_customer_id) as cnt,
            sum(optout_flg) as oo_cnt,
            cast(sum(optout_flg) as decimal(15,5))/cast(count(distinct ee_customer_id) as decimal(15,5)) as oor
            from campaign_data.atc_sccv_ids

            where brand='EE'
            and channel in ('M','E')
            and control_grp_flg = 'N'
            and campaign_objective = 'CROSS-SELL'
            -- and cast(date_format(date_of_delivery,'%Y-%m-%d') as date)  < date_parse('2020-12-31','%Y-%m-%d')
            and cast(date_format(date_of_delivery,'%Y-%m-%d') as date)  > date_parse('2020-01-01','%Y-%m-%d')
            group by 1)
        where cnt > 75000 and cnt < 2000000 and oor > 0.0001 
        ) filter_by_size
        on base.campaign_code = filter_by_size.campaign_code

    where brand='EE'
    and channel in ('M','E')
    and control_grp_flg = 'N'
    and campaign_objective = 'CROSS-SELL'
    -- and cast(date_format(date_of_delivery,'%Y-%m-%d') as date)  < date_parse('2020-12-31','%Y-%m-%d')
    and cast(date_format(date_of_delivery,'%Y-%m-%d') as date)  > date_parse('2020-01-01','%Y-%m-%d')
    
    group by ee_customer_id having sum(optout_flg) is null
) A
'''

pd.read_sql(sql_code, conn)

#QA
df = pd.read_sql('''select count(*),max(optout_flag) from campaign_data.atc_ee_not_optout_customers_model;''', conn)
df.head()

,_col0,_col1
0,6023869,None


In [8]:
pd.read_sql('''select * from campaign_data.atc_ee_not_optout_customers_model limit 100;''', conn)

,ee_customer_id,optout_flag,click_flag,optout_cnt,optout_date,day_1_week_ago,date_month
0,1151250181,None,NaN,None,2020-12-15,2020-12-09,202011
1,900094534774,None,NaN,None,2020-12-14,2020-12-08,202011
2,900103082612,None,NaN,None,2020-12-15,2020-12-09,202011
3,1106978306,None,NaN,None,2020-12-15,2020-12-09,202011
4,900110631761,None,NaN,None,2020-12-16,2020-12-10,202011
...,...,...,...,...,...,...,...
95,1085472796,None,NaN,None,2020-12-15,2020-12-09,202011
96,900104863326,None,NaN,None,2020-04-24,2020-04-18,202003
97,900121483886,None,NaN,None,2020-12-16,2020-12-10,202011
98,1146372966,None,1.0,None,2020-12-15,2020-12-09,202011


- Check the numbers so i cna have around 500k cusotmers in total nad 100k that are 1
- 500k was too much i will 300k in total, 300k was also alot i will drop it to 100k

### original weighted base

In [9]:
#Sample base of customers who have NOT opted out
%time

# IMPORTANT - DO NOT DEVIATE FROM USING optout_model_base AS IT IS REFERENCED IN MULTIPLE PLACES FORWARD

sql_code = '''drop table if exists  campaign_data.optout_model_base;'''
pd.read_sql(sql_code, conn)

sql_code = '''
CREATE TABLE IF NOT EXISTS campaign_data.optout_model_base as 

select * from (
  select * from campaign_data.atc_ee_not_optout_customers_model 
  )TABLESAMPLE BERNOULLI(100)
UNION ALL 
select * from (
  select * from campaign_data.atc_ee_optout_customers_model
  )TABLESAMPLE BERNOULLI(100)  
  '''
pd.read_sql(sql_code, conn)


CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs


,rows


In [10]:
sql_code = '''select 
                count(*) as total_rows,
                count(distinct ee_customer_id) as total_customers,
                sum(optout_flag) as total_optouts 
                from campaign_data.optout_model_base;'''

df = pd.read_sql(sql_code, conn)
df.head()

,total_rows,total_customers,total_optouts
0,6033788,6033788,9919


In [11]:
sql_code = '''select 
               *
             from campaign_data.optout_model_base where optout_flag is null limit 100'''

df = pd.read_sql(sql_code, conn)
df.head()

,ee_customer_id,optout_flag,click_flag,optout_cnt,optout_date,day_1_week_ago,date_month
0,1131746188,None,NaN,None,2020-12-14,2020-12-08,202011
1,1134772848,None,1.0,None,2020-10-30,2020-10-24,202009
2,1108553628,None,NaN,None,2020-11-18,2020-11-12,202010
3,900129776563,None,NaN,None,2020-03-18,2020-03-12,202002
4,1028804098,None,NaN,None,2020-10-30,2020-10-24,202009
